### Road Follower - Train Model

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
      
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

print("Dataset created")

#############################################
# Split dataset into train and test sets
#############################################

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

print(f"Dataset splitted into test ({test_percent}) and train ({1 - test_percent})")

#############################################
# Create data loaders to load data in batches
#############################################

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

print("Created data loaders to load data in batches")

#############################################
# Define Neural Network Model 
#############################################

# More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
# More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

model = models.resnet18(pretrained=True)

# ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1
# Finally, we transfer our model for execution on the GPU

model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

print("Neural network defined: resnet18")

#############################################
# Train Regression
#############################################

NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
        
print("Training ready")

Dataset created
Dataset splitted into test (0.1) and train (0.9)
Created data loaders to load data in batches
Neural network defined: resnet18
0.426947, 1.039655
0.079806, 0.089837
0.054553, 0.056257
0.047269, 0.030060
0.043160, 0.023554
0.041059, 0.015065
0.029143, 0.021839
0.049483, 0.034181
0.032383, 0.022728
0.034974, 0.014036
0.017988, 0.023199
0.024454, 0.017966
0.021003, 0.033634
0.030686, 0.040001
0.017982, 0.018189
0.012171, 0.017234
0.014297, 0.011828
0.013420, 0.012308
0.014426, 0.024562
0.030701, 0.035048
0.017939, 0.016087
0.008860, 0.008853
0.017333, 0.029628
0.020248, 0.017575
0.014360, 0.013838
0.010076, 0.022365
0.009960, 0.017788
0.012574, 0.013765
0.010116, 0.015311
0.016581, 0.031512
0.013772, 0.013317
0.009296, 0.010667
0.007176, 0.012913
0.007465, 0.013404
0.005676, 0.023203
0.006894, 0.015113
0.006274, 0.020054
0.006947, 0.012572
0.007686, 0.013122
0.005245, 0.014598
0.004152, 0.015647
0.004415, 0.016166
0.006659, 0.012281
0.006181, 0.013007
0.004373, 0.018895
0.